# Notebook For NLP (Natural Language Processing).
<hr>

* This Notebook is used for the process of social listening.
* This notebook processes text (unstructured data)
* Initially data brought from SentiOne


# NPL Process

In [1]:
import sys
import os

# Get the parent directory of 'topology/'
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)  # Add 'parent/' to Python's path

In [2]:
import config.m365.auth.microsoft_graph as mg

In [3]:
from config import settings, paths
from NLP_en import *

El directorio ya existe: C:/Users/Santi/Documents/atinna/repos/escuchas\Antisemitism\General\2025-02-20_2025-05-19\v2.0


## User Parameter

In [4]:
# Autenticación
token_response = mg.authenticate(client_id= settings.id_client, auth= settings.AUTHORITY, scopes= settings.GRAPH_SCOPES)
headers = mg.get_headers(token_response)

✅ Usando sesión existente para santiago.munera@atinna.co


In [5]:
f"{settings.dir_cliente}/{settings.dir_escucha}/{settings.dir_periodo}/{settings.dir_version}"

'Antisemitism/General/2025-02-20_2025-05-19/v2.0'

In [6]:
team_id = mg.get_team_id(headers, settings.dir_cliente) if headers else None
sharepoint_site = mg.get_sharepoint_site(headers, team_id)
site_id = sharepoint_site["id"]
drive_items = mg.get_drive_items(headers, site_id)
dev_folder = next((file for file in drive_items if file["name"].lower() == "desarrollo"), None)
dev_folder_id = dev_folder["id"]
dev_files = mg.get_folder_contents(headers, site_id, dev_folder_id)

In [7]:
os.makedirs(paths.paths['local']['output']['lda']['base'], exist_ok=True)
os.makedirs(paths.paths['local']['output']['lda']['models'], exist_ok=True)
os.makedirs(paths.paths['local']['output']['lda']['figures'], exist_ok=True)

In [8]:
mg.create_folder(headers, site_id, dev_folder_id, paths.paths['cloud']['output']['lda']['base'], separator="/")
id_folder = mg.get_folder_id(headers, site_id, dev_folder_id, paths.paths['cloud']['output']['lda']['base'], separator="/")

📁 Intentando crear 'General'...
📂 La carpeta 'General' ya existe.
📁 Intentando crear '2025-02-20_2025-05-19'...
📂 La carpeta '2025-02-20_2025-05-19' ya existe.
📁 Intentando crear 'v2.0'...
📂 La carpeta 'v2.0' ya existe.
📁 Intentando crear 'output'...
📂 La carpeta 'output' ya existe.
📁 Intentando crear 'lda'...
📂 La carpeta 'lda' ya existe.


In [9]:
id_folder = mg.get_folder_id(headers, site_id, dev_folder_id, paths.paths['cloud']['data']['base'], separator="/")
files = mg.get_folder_contents(headers, site_id, id_folder)
print("\n📂 Archivos en la carpeta Desarrollo:")
for file in files:
    print(f"- {file['name']} ({file['id']})")


📂 Archivos en la carpeta Desarrollo:
- AntiSemitismo_only_english.csv (01CVDJC6C4KPVXFOVD4JELNZUOGZ2JFCVO)


In [10]:
file_name = input("Por favor digite el nombre exacto del archivo: ")
file_metadata = next((file for file in files if file["name"] == file_name), None)
file_id = file_metadata["id"]
file = mg.download_file(headers, site_id, file_id,os.path.join(settings.temp_dir_path, "data"), create_missing_dirs=True)

✅ Archivo descargado exitosamente en: C:/Users/Santi/Documents/atinna/repos/escuchas\Antisemitism\General\2025-02-20_2025-05-19\v2.0\data\AntiSemitismo_only_english.csv


In [17]:
lib='pandas'#polars, pandas

#column names and column for use.
names = {
'Autor':'author',
'Contenido de la publicación':'content',#Minimum required
'Creado':'date',
'Contexto':'context',
'Link para la fuente':'link',
'Grupo de dominio':'domain_group',#Minimum required
'País':'country',
'Tipo específico':'type'
}

# Filter of values for columns
filter_list = ['Twitter','Facebook','Instagram','Video','TikTok','Reddit']

new_stop = [
    # artículos, cuantificadores, conectores muy comunes
    'a', 'an', 'the', 'and', 'or', 'but', 'if', 'so', 'yet', 'nor',
    'for', 'to', 'of', 'in', 'on', 'at', 'by', 'off', 'from', 'with',
    'without', 'into', 'onto', 'over', 'under', 'between', 'through',
    'about', 'above', 'below', 'across', 'around', 'during', 'before',
    'after', 'since', 'until', 'while', 'because', 'though', 'although',
    'unless', 'where', 'when', 'whenever', 'wherever', 'once',

    # pronombres, determinantes y sus variantes coloquiales
    'i', 'me', 'my', 'mine', 'myself',
    'you', 'u', 'ur', 'your', 'yours', 'yourself', 'yourselves',
    'he', 'him', 'his', 'himself',
    'she', 'her', 'hers', 'herself',
    'it', 'its', 'itself',
    'we', 'us', 'our', 'ours', 'ourselves',
    'they', 'them', 'their', 'theirs', 'themselves',
    'this', 'that', 'these', 'those', 'tho', 'thats', 'til',
    'some', 'any', 'none', 'each', 'either', 'neither', 'both', 'all',
    'another', 'other', 'others', 'such',

    # verbos auxiliares y formas contraídas
    'am', 'is', 'are', 'was', 'were', 'be', 'being', 'been',
    'do', 'does', 'did', 'doing',
    'have', 'has', 'had', 'having',
    'can', 'cant', "can't", 'cannot',
    'could', 'couldnt', "couldn't",
    'shall', 'should', 'shouldnt', "shouldn't",
    'will', 'wont', "won't", 'would', 'wouldnt', "wouldn't",
    'may', 'might', 'must', 'ought',
    "i'm", "you're", "he's", "she's", "it's", "we're", "they're",
    "i've", "you've", "we've", "they've",
    "i'll", "you'll", "he'll", "she'll", "we'll", "they'll",
    "i'd", "you'd", "he'd", "she'd", "we'd", "they'd",
    "ain't", "aren't", "isn't", "wasn't", "weren't", "hasn't",
    "haven't", "hadn't", "doesn't", "don't", "didn't", "won't",
    "would've", "could've", "should've", "must've", "might've",

    # números y cuantificadores escritos
    'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight',
    'nine', 'ten', 'hundred', 'thousand', 'million', 'billion',
    'first', 'second', 'third', 'fourth', 'fifth', 'last', 'next',
    'many', 'few', 'several', 'lot', 'lots', 'plenty',

    # adverbios y calificadores muy frecuentes / muletillas
    'just', 'really', 'very', 'quite', 'pretty', 'rather', 'kinda',
    'sorta', 'somehow', 'anyway', 'anyways', 'actually', 'basically',
    'literally', 'seriously', 'especially', 'probably', 'likely',
    'maybe', 'perhaps', 'almost', 'already', 'still', 'yet', 'ever',
    'never', 'always', 'often', 'sometimes', 'rarely', 'seldom',
    'soon', 'now', 'today', 'tonight', 'tomorrow', 'yesterday',
    'ago', 'here', 'there', 'everywhere', 'elsewhere',

    # interjecciones, filler words y coloquialismos
    'oh', 'ok', 'okay', 'okey', 'yo', 'hey', 'hi', 'hello', 'bye',
    'thanks', 'thank', 'thx', 'pls', 'plz', 'please', 'welcome',
    'yeah', 'yea', 'yep', 'nope', 'nah', 'uh', 'uhh', 'uhm', 'umm',
    'lol', 'lmao', 'rofl', 'omg', 'idk', 'imo', 'imho', 'btw', 'tho',
    'haha', 'hehe', 'huh', 'wow', 'whoa', 'aw', 'aww', 'ouch', 'yikes',

    # modismos de internet y abreviaturas
    'brb', 'afaik', 'irl', 'fyi', 'ftw', 'gg', 'smh', 'tbh', 'tbqh',
    'w/', 'w/o', 'b/c', 'bc', 'cuz', 'coz', 'cus', 'cause', 'ya',
    'gonna', 'wanna', 'gotta', 'lemme', 'gimme', 'outta', 'lotta',
    'kinda', 'sorta', 'shoulda', 'coulda', 'woulda', 'ain',
    'n/a', 'n\\a', 'etc', 'etc.', 'et', 'al', 'vs', 'via',

    # títulos y cortesías
    'mr', 'mrs', 'ms', 'miss', 'dr', 'prof', 'sir', 'madam', 'maam',
    'lady', 'gent', 'gents',

    # palabras genéricas y de poco valor semántico
    'thing', 'things', 'stuff', 'something', 'anything', 'nothing',
    'everything', 'place', 'places', 'way', 'ways', 'kind', 'kinds',
    'type', 'types', 'lot', 'lots', 'part', 'parts', 'bit', 'bits',
    'area', 'areas', 'case', 'cases', 'point', 'points', 'side',
    'sides', 'fact', 'facts', 'idea', 'ideas', 'problem', 'problems',
    'question', 'questions',

    # verbos comodín (ya cubiertos en gran parte por stoplists estándar,
    # pero agregamos variaciones coloquiales)
    'get', 'gets', 'getting', 'got', 'gotten',
    'take', 'takes', 'taking', 'took',
    'make', 'makes', 'making', 'made',
    'say', 'says', 'saying', 'said',
    'see', 'seen', 'seeing',
    'come', 'comes', 'coming', 'came',
    'go', 'goes', 'going', 'gone',
    'know', 'knows', 'knowing', 'knew', 'known',
    'think', 'thinks', 'thinking', 'thought',

    # variaciones ortográficas o con apóstrofes omitidos (SMS/chat)
    'dont', 'doesnt', 'isnt', 'wasnt', 'werent', 'hasnt', 'havent',
    'hadnt', 'couldnt', 'shouldnt', 'wouldnt', 'cant', 'wont', 'im',
    'ive', 'ill', 'id', 'youre', 'youve', 'youll', 'youd', 'theyre',
    'theyve', 'theyll', 'theyd', 'weve', 'well', 'wed', 'hes', 'shes',
    'its', 'thats', 'theres', 'heres', 'whats', 'lets',

    # nombres propios muy genéricos (opcionales, pero a veces se filtran)
    'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',
    'sunday', 'january', 'february', 'march', 'april', 'may', 'june',
    'july', 'august', 'september', 'october', 'november', 'december',
    'usa', 'us', 'u.s.', 'uk', 'u.k.', 'eu', 'e.u.', 'america',

    # tokens misceláneos y de puntuación
    '.', ',', ';', ':', '!', '?', '…', '-', '–', '—', '(', ')',
    '[', ']', '{', '}', '«', '»', '"', "'", '`', '´', '“', '”', '‘', '’',
    '&', '%', '$', '#', '@', '©', '®', '™', '+', '=', '/', '\\'
]


sample_frac=None#0.25

n_volume=None#10000

optimization = True

min_topic = 4#2

max_topic = 10#2

k=None

sep=';'

## Processing

In [18]:
%%time
#the format of the data can changue. A firs view for selectin parameters.
answer = input('Do you require the first description of the data?? yes or NO')
if answer.lower()=='yes':
    df = local_reading(path=file,sep=sep,lib=lib)
    x = df['Grupo de dominio'].unique()
    print(f'Data size:{df.shape}')
    print(f'columns :{df.columns}')
    print(f'Available social networks: {x}')
    del df, x, answer
else:
    print('No need for the initial description')

Data size:(821056, 44)
columns :Index(['id', 'Tipo específico', 'Título', 'Autor', 'Id del Author',
       'Contenido de la publicación', 'Creado', 'Agregar al sistema',
       'Contexto', 'Link para la fuente', 'Dominio', 'Sentimiento',
       'Puntos de sentimiento', 'Grupo de dominio', 'Etiqueta',
       'Palabras Clave', 'Género', 'Nombre del proyecto', 'Unnamed: 18',
       'influenceScore', 'comments', 'views', 'shares', 'wow', 'love', 'like',
       'haha', 'sad', 'angry', 'thankful', 'uniqueVievs', 'fans',
       'Categoría de página de Facebook', 'retweet', 'favs', 'hearts', 'likes',
       'dislikes', 'followers', 'Geolocalización', 'Lenguaje', 'País',
       'Calificación', 'ID del hilo'],
      dtype='object')
Available social networks: ['Portales' 'Twitter' 'Facebook' 'Forums' 'Reddit' 'Blogs' 'Reseñas']
CPU times: total: 17.3 s
Wall time: 19.1 s


In [19]:
%%time
Social_listening_data_process_and_modeling_LDA(
    sep=sep,
    path_data=file,
    path_output= paths.paths['local']['output']['lda']['base'],
    path_output_model= paths.paths['local']['output']['lda']['models'],
    lib=lib,
    column_names=names,
    filter_list=filter_list,
    new_stop=new_stop,
    sample_frac=sample_frac,
    n_volume=n_volume,
    optimization =optimization,
    min_topics=min_topic, 
    max_topics=max_topic,
    step_size=1,
    k=k,
    n_cpu=6,
    n_cpu_lda=8
)

Start Process LDA
Stopwords have been downloaded.
*Reading
Size Of the Data:(821056, 44)
Reading --- 17.08980894088745 seconds ---
*Normalize columns
Is pandas Dataframe
Normalize columns --- 0.17403364181518555 seconds ---
*Network distribution
Network distribution --- 0.06351327896118164 seconds ---
*Drop nan
Is pandas Dataframe
Drop nan --- 0.10552978515625 seconds ---
*Repeated post
Repeated post --- 15.310202598571777 seconds ---
*Hash tags table
Hash tags table --- 0.6215035915374756 seconds ---
*Drop duplicates
Is pandas Dataframe
Drop duplicates --- 0.238602876663208 seconds ---
Size Of the Data after cleaning:(754655, 8)
*Set data types
Set data types --- 0.18492865562438965 seconds ---
*Filter columns of social network
['Twitter' 'Facebook' 'Reddit']
Filter columns of social network --- 0.7569177150726318 seconds ---
*Text cleaning paralelized
Is pandas Dataframe


c:\Users\Santi\miniconda3\envs\social-analysis\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Text cleaning paralelized --- 51.669341802597046 seconds ---
*Transform list into string
Is pandas Dataframe


c:\Users\Santi\miniconda3\envs\social-analysis\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


from list to str --- 69.17955803871155 seconds ---
*Drop duplicates after cleaning
Is pandas Dataframe
Drop duplicates after cleaning --- 0.33431100845336914 seconds ---
Size Of the Data after cleaning:(532824, 9)
*From list to string
Is pandas Dataframe


...

text procesing --- 361.7792537212372 seconds ---
*Modeling LDA


LDA different k: 100%|██████████| 6/6 [2:16:42<00:00, 1367.11s/it]


*Modeling LDA --- 8218.826862335205 seconds ---
*Joing words
Is pandas Dataframe


c:\Users\Santi\miniconda3\envs\social-analysis\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Joing words --- 12.978819131851196 seconds ---
End of process --- 8750.989723920822 seconds ---
None
CPU times: total: 2h 21min 15s
Wall time: 2h 25min 51s


# Outputs

## Metadata
Contains information about the process.

In [20]:
path_metadata = os.path.join(paths.paths['local']['output']['lda']['base'],"metadata.json")
metadata = json_read(path_metadata)
metadata

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Santi/Documents/atinna/repos/escuchas\\Antisemitism\\General\\2025-02-20_2025-05-19\\v2.0\\lda\\metadata.json'

## Hashtags frequency.

In [ ]:
[i for i in os.listdir(paths.paths['local']['output']['lda']['base']) if 'Hash'in i]

['Hash_global_pandas.csv']

In [ ]:
hash= [local_reading(path=paths.paths['local']['output']['lda']['base']+i) for i in os.listdir(paths.paths['local']['output']['lda']['base']) if 'Hash'in i]
hash[0]

## Most repeated posts.

In [ ]:
# Repe= [local_reading(path=paths['local']['output']['base']+i) for i in os.listdir(paths['local']['output']['base']) if 'Repea'in i]
# df_repe = Repe[0].head(15).set_index('content')
# df_repe [df_repe['count']>1]

## Network Distribution.

In [ ]:
Netw= [local_reading(path=paths.paths['local']['output']['lda']['base']+i) for i in os.listdir(paths.paths['local']['output']['lda']['base']) if 'net'in i]
Netw

[None]

# Sync

In [ ]:
mg.upload_folder(headers, site_id, dev_folder_id, paths.paths['local']['output']['lda']['base'], paths.paths['cloud']['output']['lda']['base'], separator="/", conflict_behavior="fail", skip_on_error=False)

❌ Usuario no autenticado o token expirado.
